# Setting Up

In [ ]:
#Install required packages 
!wget  -q https://raw.githubusercontent.com/ammarnasr/land-classification/main/requirements.txt
!pip install -q -r requirements.txt

In [ ]:
#Dwonload data labels
!wget -q https://github.com/ammarnasr/land-classification/raw/main/data/joblibs/labels.joblib

Joblib is library for saving and loading Python objects to/from disk (https://joblib.readthedocs.io/en/stable/)

# Exploring the Labels

In [ ]:
import joblib
import geopandas as gpd
labels_gdf = joblib.load('labels.joblib')
labels_gdf = labels_gdf.to_crs(crs='EPSG:4326') # convert to WGS84 coordinate system
labels_gdf.head(1)

The labels are saved as a GeoDataFrame (https://geopandas.org/) which is a special type of DataFrame that has a geometry column. The geometry column is a special column that contains the coordinates of the label. The geometry column is used to plot the labels on the map.

Each row in the labels GeoDataFrame contains many columns. The most important columns are:
- Rainfed: whether the field is rainfed or irrigated (1 = rainfed, 0 = irrigated)
- Crop_Type: the crop type or uncultivated
- Year: the year the field was labeled
- geometry: the coordinates of the field
- State: the state the field is located in
- Area_M2: the area of the field in square meters

In [ ]:
import folium
m = folium.Map(location=[14.18, 33.09], zoom_start=12)
labels_gdf.explore(column= 'Crop_Type', tooltip= ['Crop_Type', 'State', 'Rainfed', 'Area_M2'], m = m, legend=False)
m

Folium is a library for plotting data on interactive maps that is used with GeoPandas to plot the labels on the map (https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html)

# Getting Satellite Imagery

In [ ]:
!wget -q https://raw.githubusercontent.com/ammarnasr/land-classification/main/senHub.py # Class to interact with Sentinel Hub API
!wget -q https://raw.githubusercontent.com/ammarnasr/land-classification/main/new_app.py # Wrapper around senHub.py for simple use
!wget -q https://raw.githubusercontent.com/ammarnasr/land-classification/main/new_utils.py # Utility functions

Sentinel-2 is a satellite that provide free 10-meter resolution imagery of the entire world every 5 days. The Sentinel-2 imagery consist of 13 different bands inculding visible, near-infrared, and shortwave infrared bands. The SentinelHub Python package is used to download the imagery by specifying the coordinates of the field, the date of the image, and the bands to download (https://sentinelhub-py.readthedocs.io/en/latest/index.html)

In [ ]:
import os
os.makedirs('scripts', exist_ok=True)

all_bands_script = """

    //VERSION=3
    function setup() {
      return {
          input: [{
              bands: ["B01", "B02", "B03","B04","B05","B06","B07","B08","B8A","B09","B10", "B11", "B12"],
              units: ["REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE", "REFLECTANCE"]

          }],
          output: {
              bands: 13,
              sampleType: "FLOAT32" // 32-bit floating point
          }
      };
  }

  function evaluatePixel(sample) {
      return [sample.B01,
              sample.B02,
              sample.B03,
              sample.B04,
              sample.B05,
              sample.B06,
              sample.B07,
              sample.B08,
              sample.B8A,
              sample.B09,
              sample.B10,
              sample.B11,
              sample.B12];
  }
"""

with open('scripts/all_.js', 'w') as f:
    f.write(all_bands_script)


true_color_script = """
//VERSION=3

function setup() {
  return {
      input: [{
          bands: ["B02", "B03", "B04"]
      }],
      output: {
          bands: 3
      }
  };
}

function evaluatePixel(sample) {
  return [sample.B04, sample.B03, sample.B02];
}
"""

with open('scripts/true_color.js', 'w') as f:
    f.write(true_color_script)


Bands are selected based on scripts written in JavaScript and passed to Sentinel Hub API as the evalscript parameter (https://custom-scripts.sentinel-hub.com/custom-scripts/)


In [ ]:
# Filter the labels to get the boundries of fields in a specific state
print(labels_gdf.State.unique()) 
state = 'Gaziera'
state_gdf = labels_gdf[labels_gdf.State == state] 
state_gdf.head()

# Get the total boundries of the fields in the state to be used in the API request
state_gdf = state_gdf.to_crs(crs='EPSG:4326')
total_bounds = state_gdf.total_bounds

#Visualize the total boundries with the fields boundries
import folium
m = folium.Map(location=[14.09, 33.56], zoom_start=12)
state_gdf.explore(column= 'Crop_Type', tooltip= ['Crop_Type', 'State', 'Rainfed'], m = m, legend=False)
total_bounds = [(total_bounds[1], total_bounds[0]), (total_bounds[3], total_bounds[2])]
folium.Rectangle(bounds=total_bounds, color='red').add_to(m)
m

In [ ]:
import new_app
import shapely.geometry

#Set the boundries of the image to be downloaded, the date and the evalscript to be passed to the API
total_polygon = shapely.geometry.box(*total_bounds, ccw=True)
date = '2021-10-29'
location_name = 'Gaziera'
evalscript_all_bands = 'all'
evalscript_true_color = 'true_color'

#Get the images from the API as numpy arrays and the path in which they are saved as GeoTIFF files
true_color_image, true_color_save_path = new_app.get_any_image_from_sentinelhub(
    polygon=total_polygon,
    date=date,
    location=location_name,
    evalscript=evalscript_true_color,
    )

all_bands_image, all_bands_save_path = new_app.get_any_image_from_sentinelhub(
    polygon=total_polygon,
    date=date,
    location=location_name,
    evalscript=evalscript_all_bands,
    )